In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from datetime import datetime
import os
import unidecode
from scipy import stats

In [2]:
#Funções para tratamento de registro com espaços desnecessário
def funcLimparEspacos(df,col):
    palavras = df[col]
    palavras = palavras.strip() 
    return palavras


def removerAcentuacaoCidade(df,col):
    cidade = unidecode.unidecode(df[col])
    return cidade

In [3]:
#Lendo arquivo CSV com registro dos acidentes no ano de 2017
dataset_2017 = pd.read_csv('csv/datatran2017.csv',encoding='ISO-8859-1',sep=";")

In [4]:
#Padronizando o dataframe com os dados relevantes para analise
dataset_2017 = dataset_2017.drop(dataset_2017.columns[[0,3,5,6,9,10,11,12,14,15,16,19,20,21,22,24,25,26,27,28,29]], axis=1)

In [5]:
#Filtrando os acidentes ocorridos no Estado de PE
dataset_2017 = dataset_2017[(dataset_2017.uf == "PE")]

In [6]:
#Padronizando o formato de data
dataset_2017.data_inversa=pd.to_datetime(dataset_2017.data_inversa).dt.strftime('%Y-%m-%d')
dataset_2017.rename(columns={'data_inversa':'Date'}, inplace=True)

In [7]:
#----Tratamento na Base - Limpeza no excesso de espaços nos resgistros de municipio e causa_acidente ----
dataset_2017['municipio'] = dataset_2017.apply(funcLimparEspacos,col = 3, axis=1)
dataset_2017['causa_acidente'] = dataset_2017.apply(funcLimparEspacos,col = 4, axis=1)

In [8]:
#Separacao da data
dataset_2017['year'] = pd.DatetimeIndex(dataset_2017['Date']).year
dataset_2017['month'] = pd.DatetimeIndex(dataset_2017['Date']).month
dataset_2017['day'] = pd.DatetimeIndex(dataset_2017['Date']).day

#Contador unitario para cada acidente
dataset_2017['acidente'] = 1

In [9]:
#Lendo arquivo CSV com registro de precipitação em PE
dataset_Precipitacao = pd.read_csv('csv/result_monthly_precipitation_sum.csv')

In [10]:
#Filtrando o Municipio de RECIFE do Dataframe de acidentes
dataset_2017_Recife = dataset_2017[(dataset_2017.municipio == "RECIFE")]

In [11]:
#Filtrando ano de 2016 da Base de Precipitação e o Municipio de Recife
dataset_Precipitacao_2017 = dataset_Precipitacao[(dataset_Precipitacao.year == 2017)]
dataset_Precipitacao_2017_Recife = dataset_Precipitacao_2017[(dataset_Precipitacao_2017.Estacao == "RECIFE")]
dataset_Precipitacao_2017_Recife.rename(columns={'Estacao':'municipio'}, inplace=True)

c:\users\carlos\appdata\local\programs\python\python36-32\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [17]:
#Filtrando os acidentes causados por Ingestão de Álcool
dataset_2017_Recife_Acidentes_alcool = dataset_2017_Recife[(dataset_2017_Recife.causa_acidente == "Ingestão de Álcool")]

#Aplicando somatorio mensal dos acidentes causados por Ingestão de Álcool
dataset_2017_Recife_Acidentes_alcool_somados = pd.DataFrame({'soma_acidentes_mes' : dataset_2017_Recife_Acidentes_alcool.groupby(["municipio", "month"])["acidente"].sum()}).reset_index()
df_precipitacao_x_Acidentes_alcool_somados = pd.merge(dataset_Precipitacao_2017_Recife, dataset_2017_Recife_Acidentes_alcool_somados, on = ('municipio','month'))

In [18]:
df_precipitacao_x_Acidentes_alcool_somados['Precipitacao'].corr(df_precipitacao_x_Acidentes_alcool_somados['soma_acidentes_mes'], method='spearman')

0.0747574318811214

In [21]:
#Filtrando os acidentes causados pela falta de atenção do condutor
dataset_2017_Recife_Acidentes_condutor_desatento = dataset_2017_Recife[(dataset_2017_Recife.causa_acidente == "Falta de Atenção à Condução")]

#Aplicando somatorio mensal dos acidentes causados pela falta de atenção do condutor
dataset_2017_Recife_Acidentes_condutor_desatento_somados = pd.DataFrame({'soma_acidentes_mes' : dataset_2017_Recife_Acidentes_condutor_desatento.groupby(["municipio", "month"])["acidente"].sum()}).reset_index()
dataset_2017_Recife_Acidentes_condutor_desatento_somados = pd.merge(dataset_Precipitacao_2017_Recife, dataset_2017_Recife_Acidentes_condutor_desatento_somados, on = ('municipio','month'))

In [22]:
#Coeficiente de Correlação Spearman 
dataset_2017_Recife_Acidentes_condutor_desatento_somados['Precipitacao'].corr(dataset_2017_Recife_Acidentes_condutor_desatento_somados['soma_acidentes_mes'], method='spearman')

0.25089748992277794

In [25]:
#Filtrando os acidentes causados pela falta de distancia de segura entre os veiculos
dataset_2017_Recife_Acidentes_condutor_distancia = dataset_2017_Recife[(dataset_2017_Recife.causa_acidente == "Não guardar distância de segurança")]

#Aplicando somatorio mensal dos acidentes causados pela falta de distancia de segura entre os veiculos
dataset_2017_Recife_Acidentes_condutor_distancia_somados = pd.DataFrame({'soma_acidentes_mes' : dataset_2017_Recife_Acidentes_condutor_distancia.groupby(["municipio", "month"])["acidente"].sum()}).reset_index()
dataset_2017_Recife_Acidentes_condutor_distancia_somados = pd.merge(dataset_Precipitacao_2017_Recife, dataset_2017_Recife_Acidentes_condutor_distancia_somados, on = ('municipio','month'))

In [26]:
#Coeficiente de Correlação Spearman 
dataset_2017_Recife_Acidentes_condutor_distancia_somados['Precipitacao'].corr(dataset_2017_Recife_Acidentes_condutor_distancia_somados['soma_acidentes_mes'], method='spearman')

0.24956410143983132

In [32]:
#Filtrando os acidentes causados por defeitos na via
dataset_2017_Recife_Acidentes_defeito_via = dataset_2017_Recife[(dataset_2017_Recife.causa_acidente == "Defeito na Via")]

#Aplicando somatorio mensal dos acidentes causados por defeitos na via
dataset_2017_Recife_Acidentes_defeito_via_somados = pd.DataFrame({'soma_acidentes_mes' : dataset_2017_Recife_Acidentes_defeito_via.groupby(["municipio", "month"])["acidente"].sum()}).reset_index()
dataset_2017_Recife_Acidentes_defeito_via_somados = pd.merge(dataset_Precipitacao_2017_Recife, dataset_2017_Recife_Acidentes_defeito_via_somados, on = ('municipio','month'))

In [33]:
#Coeficiente de Correlação Spearman 
dataset_2017_Recife_Acidentes_defeito_via_somados['Precipitacao'].corr(dataset_2017_Recife_Acidentes_defeito_via_somados['soma_acidentes_mes'], method='spearman')

0.5611425419454066

In [39]:
#Filtrando os acidentes causados por animais na via
dataset_2017_Recife_Acidentes_animais_pista = dataset_2017_Recife[(dataset_2017_Recife.causa_acidente == "Animais na Pista")]

#Aplicando somatorio mensal dos acidentes causados por defeitos na via
dataset_2017_Recife_Acidentes_animais_pista_somados = pd.DataFrame({'soma_acidentes_mes' : dataset_2017_Recife_Acidentes_animais_pista.groupby(["municipio", "month"])["acidente"].sum()}).reset_index()
dataset_2017_Recife_Acidentes_animais_pista_somados = pd.merge(dataset_Precipitacao_2017_Recife, dataset_2017_Recife_Acidentes_animais_pista_somados, on = ('municipio','month'))

In [40]:
#Coeficiente de Correlação Spearman 
dataset_2017_Recife_Acidentes_animais_pista_somados['Precipitacao'].corr(dataset_2017_Recife_Acidentes_animais_pista_somados['soma_acidentes_mes'], method='spearman')

0.31139957766460924